In [ ]:
import pyspark
import findspark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

In [ ]:
spark

In [ ]:
arquivo = "files\programacoes.json"

In [ ]:
df = spark.read.format('org.apache.spark.sql.json') \
        .load(arquivo)

In [ ]:
df = spark.read\
.option("multiline", "true")\
.json(arquivo)
display(df)

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [ ]:
# Flatten array of structs an structs

def flatten(df):
    # Compute complex filds (lists and structs) in schema
    complex_fields = dict([(field.name, field.dataType)
			for field in df.schema.fields
			if type(field.dataType) == ArrayType or type(field.dataType) == StructType])
    while len(complex_fields)!=0:
        col_name = list(complex_fields.keys())[0]
        print ("Processing :"+col_name+" Type : "+str(type(complex_fields[col_name])))

    # if structType then convert all sub element to columns.
    # i.e. flatten structs
    if (type(complex_fields[col_name]) == StructType):
        expanded = [col(col_name+'.'+k).alias(col_name+'_'+k) for k in [n.name for n in complex_fields[col_name]]]
        df = df.select("*", *expanded).drop(col_name)

    # if arrayType then add the array elements as rows using the explode function
    # i.e. explede arrays
    elif (type(complex_fields[col_name]) == ArrayType):
        df = df.withColumn(col_name, explode_outer(col_name))

    # Recompute remaining complex fields in schema
    complex_fields = dict([(field.name, field.dataType)
			for field in df.schema.fields
			if type(field.dataType) == ArrayType or type(field.dataType) == StructType])
    return df

In [ ]:
df_flatten = flatten(df)